# 批标准化
在我们正式进入模型的构建和训练之前，我们会先讲一讲数据预处理和批标准化，因为模型训练并不容易，特别是一些非常复杂的模型，并不能非常好的训练得到收敛的结果，所以对数据增加一些预处理，同时使用批标准化能够得到非常好的收敛结果，这也是卷积网络能够训练到非常深的层的一个重要原因。

## 数据预处理
目前数据预处理最常见的方法就是中心化和标准化，中心化相当于修正数据的中心位置，实现方法非常简单，就是在每个特征维度上减去对应的均值，最后得到 0 均值的特征。标准化也非常简单，在数据变成 0 均值之后，为了使得不同的特征维度有着相同的规模，可以除以标准差近似为一个标准正态分布，也可以依据最大值和最小值将其转化为 -1 ~ 1 之间

这两种方法非常的常见，如果你还记得，前面我们在神经网络的部分就已经使用了这个方法实现了数据标准化，至于另外一些方法，比如 PCA 或者 白噪声已经用得非常少了。

## Batch Normalization
前面在数据预处理的时候，我们尽量输入特征不相关且满足一个标准的正态分布，这样模型的表现一般也较好。但是对于很深的网路结构，网路的非线性层会使得输出的结果变得相关，且不再满足一个标准的 N(0, 1) 的分布，甚至输出的中心已经发生了偏移，这对于模型的训练，特别是深层的模型训练非常的困难。

所以在 2015 年一篇论文提出了这个方法，批标准化，简而言之，就是对于每一层网络的输出，对其做一个归一化，使其服从标准的正态分布，这样后一层网络的输入也是一个标准的正态分布，所以能够比较好的进行训练，加快收敛速度。

batch normalization 的实现非常简单，对于给定的一个 batch 的数据 $B = \{x_1, x_2, \cdots, x_m\}$算法的公式如下

$$
\mu_B = \frac{1}{m} \sum_{i=1}^m x_i
$$
$$
\sigma^2_B = \frac{1}{m} \sum_{i=1}^m (x_i - \mu_B)^2
$$
$$
\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma^2_B + \epsilon}}
$$
$$
y_i = \gamma \hat{x}_i + \beta
$$

第一行和第二行是计算出一个 batch 中数据的均值和方差，接着使用第三个公式对 batch 中的每个数据点做标准化，$\epsilon$ 是为了计算稳定引入的一个小的常数，通常取 $10^{-5}$，最后利用权重修正得到最后的输出结果，非常的简单，下面我们可以实现一下简单的一维的情况，也就是神经网络中的情况

In [1]:
import sys
sys.path.append('..')

import torch

In [2]:
def simple_batch_norm_1d(x, gamma, beta):
    eps = 1e-5
    x_mean = torch.mean(x, dim=0, keepdim=True) # 保留维度进行 broadcast
    x_var = torch.mean((x - x_mean) ** 2, dim=0, keepdim=True)
    x_hat = (x - x_mean) / torch.sqrt(x_var + eps)
    return gamma.view_as(x_mean) * x_hat + beta.view_as(x_mean)

我们来验证一下是否对于任意的输入，输出会被标准化

In [7]:
x = torch.arange(15).view(5, 3)
gamma = torch.ones(x.shape[1])
beta = torch.zeros(x.shape[1])
print('before bn: ')
print(x)
y = simple_batch_norm_1d(x*1.0, gamma, beta)
print('after bn: ')
print(y)

before bn: 
tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11],
        [12, 13, 14]])
after bn: 
tensor([[-1.4142, -1.4142, -1.4142],
        [-0.7071, -0.7071, -0.7071],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.7071,  0.7071,  0.7071],
        [ 1.4142,  1.4142,  1.4142]])


可以看到这里一共是 5 个数据点，三个特征，每一列表示一个特征的不同数据点，使用批标准化之后，每一列都变成了标准的正态分布

这个时候会出现一个问题，就是测试的时候该使用批标准化吗？

答案是肯定的，因为训练的时候使用了，而测试的时候不使用肯定会导致结果出现偏差，但是测试的时候如果只有一个数据集，那么均值不就是这个值，方差为 0 吗？这显然是随机的，所以测试的时候不能用测试的数据集去算均值和方差，而是用训练的时候算出的移动平均均值和方差去代替

下面我们实现以下能够区分训练状态和测试状态的批标准化方法

In [8]:
def batch_norm_1d(x, gamma, beta, is_training, moving_mean, moving_var, moving_momentum=0.1):
    eps = 1e-5
    x_mean = torch.mean(x, dim=0, keepdim=True) # 保留维度进行 broadcast
    x_var = torch.mean((x - x_mean) ** 2, dim=0, keepdim=True)
    if is_training:
        x_hat = (x - x_mean) / torch.sqrt(x_var + eps)
        moving_mean[:] = moving_momentum * moving_mean + (1. - moving_momentum) * x_mean
        moving_var[:] = moving_momentum * moving_var + (1. - moving_momentum) * x_var
    else:
        x_hat = (x - moving_mean) / torch.sqrt(moving_var + eps)
    return gamma.view_as(x_mean) * x_hat + beta.view_as(x_mean)

下面我们使用上一节课将的深度神经网络分类 mnist 数据集的例子来试验一下批标准化是否有用

In [9]:
import numpy as np
from torchvision.datasets import mnist # 导入 pytorch 内置的 mnist 数据
from torch.utils.data import DataLoader
from torch import nn
from torch.autograd import Variable

In [20]:
# 使用内置函数下载 mnist 数据集
train_set = mnist.MNIST('./data', download = True, train=True)
test_set = mnist.MNIST('./data',  train=False)

def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 数据预处理，标准化
    x = x.reshape((-1,)) # 拉平
    x = torch.from_numpy(x)
    return x

train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True) # 重新载入数据集，申明定义的数据变换
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [21]:
class multi_network(nn.Module):
    def __init__(self):
        super(multi_network, self).__init__()
        self.layer1 = nn.Linear(784, 100)
        self.relu = nn.ReLU(True)
        self.layer2 = nn.Linear(100, 10)
        
        self.gamma = nn.Parameter(torch.randn(100))
        self.beta = nn.Parameter(torch.randn(100))
        
        self.moving_mean = Variable(torch.zeros(100))
        self.moving_var = Variable(torch.zeros(100))
        
    def forward(self, x, is_train=True):
        x = self.layer1(x)
        x = batch_norm_1d(x, self.gamma, self.beta, is_train, self.moving_mean, self.moving_var)
        x = self.relu(x)
        x = self.layer2(x)
        return x

In [22]:
net = multi_network()

In [23]:
# 定义 loss 函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-1) # 使用随机梯度下降，学习率 0.1

为了方便，训练函数已经定义在外面的 utils.py 中，跟前面训练网络的操作是一样的，感兴趣的同学可以去看看

In [32]:
from datetime import datetime

import torch
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable


def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().data
    return num_correct / total


def train(net, train_data, valid_data, num_epochs, optimizer, criterion):
    if torch.cuda.is_available():
        net = net.cuda()
    prev_time = datetime.now()
    for epoch in range(num_epochs):
        train_loss = 0
        train_acc = 0
        net = net.train()
        for im, label in train_data:
            if torch.cuda.is_available():
                im = Variable(im.cuda())  # (bs, 3, h, w)
                label = Variable(label.cuda())  # (bs, h, w)
            else:
                im = Variable(im)
                label = Variable(label)
            # forward
            output = net(im)
            loss = criterion(output, label)
            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.data
            train_acc += get_acc(output, label)

        cur_time = datetime.now()
        h, remainder = divmod((cur_time - prev_time).seconds, 3600)
        m, s = divmod(remainder, 60)
        time_str = "Time %02d:%02d:%02d" % (h, m, s)
        if valid_data is not None:
            valid_loss = 0
            valid_acc = 0
            net = net.eval()
            for im, label in valid_data:
                if torch.cuda.is_available():
                    im = Variable(im.cuda(), volatile=True)
                    label = Variable(label.cuda(), volatile=True)
                else:
                    im = Variable(im, volatile=True)
                    label = Variable(label, volatile=True)
                output = net(im)
                loss = criterion(output, label)
                valid_loss += loss.data
                valid_acc += get_acc(output, label)
            epoch_str = (
                "Epoch %d. Train Loss: %f, Train Acc: %f, Valid Loss: %f, Valid Acc: %f, "
                % (epoch, train_loss / len(train_data),
                   train_acc / len(train_data), valid_loss / len(valid_data),
                   valid_acc / len(valid_data)))
        else:
            epoch_str = ("Epoch %d. Train Loss: %f, Train Acc: %f, " %
                         (epoch, train_loss / len(train_data),
                          train_acc / len(train_data)))
        prev_time = cur_time
        print(epoch_str + time_str)


def conv3x3(in_channel, out_channel, stride=1):
    return nn.Conv2d(
        in_channel, out_channel, 3, stride=stride, padding=1, bias=False)


class residual_block(nn.Module):
    def __init__(self, in_channel, out_channel, same_shape=True):
        super(residual_block, self).__init__()
        self.same_shape = same_shape
        stride = 1 if self.same_shape else 2

        self.conv1 = conv3x3(in_channel, out_channel, stride=stride)
        self.bn1 = nn.BatchNorm2d(out_channel)

        self.conv2 = conv3x3(out_channel, out_channel)
        self.bn2 = nn.BatchNorm2d(out_channel)
        if not self.same_shape:
            self.conv3 = nn.Conv2d(in_channel, out_channel, 1, stride=stride)

    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(self.bn1(out), True)
        out = self.conv2(out)
        out = F.relu(self.bn2(out), True)

        if not self.same_shape:
            x = self.conv3(x)
        return F.relu(x + out, True)


class resnet(nn.Module):
    def __init__(self, in_channel, num_classes, verbose=False):
        super(resnet, self).__init__()
        self.verbose = verbose

        self.block1 = nn.Conv2d(in_channel, 64, 7, 2)

        self.block2 = nn.Sequential(
            nn.MaxPool2d(3, 2), residual_block(64, 64), residual_block(64, 64))

        self.block3 = nn.Sequential(
            residual_block(64, 128, False), residual_block(128, 128))

        self.block4 = nn.Sequential(
            residual_block(128, 256, False), residual_block(256, 256))

        self.block5 = nn.Sequential(
            residual_block(256, 512, False),
            residual_block(512, 512), nn.AvgPool2d(3))

        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.block1(x)
        if self.verbose:
            print('block 1 output: {}'.format(x.shape))
        x = self.block2(x)
        if self.verbose:
            print('block 2 output: {}'.format(x.shape))
        x = self.block3(x)
        if self.verbose:
            print('block 3 output: {}'.format(x.shape))
        x = self.block4(x)
        if self.verbose:
            print('block 4 output: {}'.format(x.shape))
        x = self.block5(x)
        if self.verbose:
            print('block 5 output: {}'.format(x.shape))
        x = x.view(x.shape[0], -1)
        x = self.classifier(x)
        return x


In [33]:
train(net, train_data, test_data, 10, optimizer, criterion)

<ipython-input-32-2d866a79c2c1>:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im, volatile=True)
<ipython-input-32-2d866a79c2c1>:56: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label, volatile=True)


Epoch 0. Train Loss: 0.296794, Train Acc: 0.915412, Valid Loss: 0.181497, Valid Acc: 0.949565, Time 00:00:09
Epoch 1. Train Loss: 0.170104, Train Acc: 0.950410, Valid Loss: 0.152157, Valid Acc: 0.955301, Time 00:00:11
Epoch 2. Train Loss: 0.131981, Train Acc: 0.961770, Valid Loss: 0.131233, Valid Acc: 0.962718, Time 00:00:11
Epoch 3. Train Loss: 0.108266, Train Acc: 0.968667, Valid Loss: 0.109220, Valid Acc: 0.969739, Time 00:00:10
Epoch 4. Train Loss: 0.094336, Train Acc: 0.971748, Valid Loss: 0.101818, Valid Acc: 0.970530, Time 00:00:11
Epoch 5. Train Loss: 0.083558, Train Acc: 0.975663, Valid Loss: 0.098579, Valid Acc: 0.971123, Time 00:00:10
Epoch 6. Train Loss: 0.073992, Train Acc: 0.978145, Valid Loss: 0.095839, Valid Acc: 0.972013, Time 00:00:10
Epoch 7. Train Loss: 0.067372, Train Acc: 0.979894, Valid Loss: 0.092314, Valid Acc: 0.972903, Time 00:00:11
Epoch 8. Train Loss: 0.062005, Train Acc: 0.981410, Valid Loss: 0.093699, Valid Acc: 0.973200, Time 00:00:11
Epoch 9. Train Loss

这里的 $\gamma$ 和 $\beta$ 都作为参数进行训练，初始化为随机的高斯分布，`moving_mean` 和 `moving_var` 都初始化为 0，并不是更新的参数，训练完 10 次之后，我们可以看看移动平均和移动方差被修改为了多少

In [34]:
# 打出 moving_mean 的前 10 项
print(net.moving_mean[:10])

tensor([-3.2891,  0.0460, -3.0133, -0.1405,  0.1346,  0.3135,  0.6890,  0.1493,
         0.2242,  0.7343], grad_fn=<SliceBackward0>)


可以看到，这些值已经在训练的过程中进行了修改，在测试过程中，我们不需要再计算均值和方差，直接使用移动平均和移动方差即可

作为对比，我们看看不使用批标准化的结果

In [35]:
no_bn_net = nn.Sequential(
    nn.Linear(784, 100),
    nn.ReLU(True),
    nn.Linear(100, 10)
)

optimizer = torch.optim.SGD(no_bn_net.parameters(), 1e-1) # 使用随机梯度下降，学习率 0.1
train(no_bn_net, train_data, test_data, 10, optimizer, criterion)

<ipython-input-32-2d866a79c2c1>:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im, volatile=True)
<ipython-input-32-2d866a79c2c1>:56: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label, volatile=True)


Epoch 0. Train Loss: 0.393515, Train Acc: 0.876599, Valid Loss: 0.237481, Valid Acc: 0.924842, Time 00:00:08
Epoch 1. Train Loss: 0.176403, Train Acc: 0.946962, Valid Loss: 0.137886, Valid Acc: 0.956883, Time 00:00:09
Epoch 2. Train Loss: 0.130885, Train Acc: 0.961154, Valid Loss: 0.116617, Valid Acc: 0.964003, Time 00:00:10
Epoch 3. Train Loss: 0.106716, Train Acc: 0.967934, Valid Loss: 0.108320, Valid Acc: 0.966475, Time 00:00:09
Epoch 4. Train Loss: 0.092154, Train Acc: 0.971665, Valid Loss: 0.101190, Valid Acc: 0.968750, Time 00:00:10
Epoch 5. Train Loss: 0.080068, Train Acc: 0.975430, Valid Loss: 0.099193, Valid Acc: 0.969640, Time 00:00:09
Epoch 6. Train Loss: 0.070579, Train Acc: 0.978395, Valid Loss: 0.090726, Valid Acc: 0.970827, Time 00:00:10
Epoch 7. Train Loss: 0.064693, Train Acc: 0.979911, Valid Loss: 0.108079, Valid Acc: 0.965190, Time 00:00:09
Epoch 8. Train Loss: 0.057960, Train Acc: 0.981826, Valid Loss: 0.098906, Valid Acc: 0.970134, Time 00:00:10
Epoch 9. Train Loss

可以看到虽然最后的结果两种情况一样，但是如果我们看前几次的情况，可以看到使用批标准化的情况能够更快的收敛，因为这只是一个小网络，所以用不用批标准化都能够收敛，但是对于更加深的网络，使用批标准化在训练的时候能够很快地收敛

从上面可以看到，我们自己实现了 2 维情况的批标准化，对应于卷积的 4 维情况的标准化是类似的，只需要沿着通道的维度进行均值和方差的计算，但是我们自己实现批标准化是很累的，pytorch 当然也为我们内置了批标准化的函数，一维和二维分别是 `torch.nn.BatchNorm1d()` 和 `torch.nn.BatchNorm2d()`，不同于我们的实现，pytorch 不仅将 $\gamma$ 和 $\beta$ 作为训练的参数，也将 `moving_mean` 和 `moving_var` 也作为参数进行训练

下面我们在卷积网络下试用一下批标准化看看效果

In [36]:
def data_tf(x):
    x = np.array(x, dtype='float32') / 255
    x = (x - 0.5) / 0.5 # 数据预处理，标准化
    x = torch.from_numpy(x)
    x = x.unsqueeze(0)
    return x

train_set = mnist.MNIST('./data', train=True, transform=data_tf, download=True) # 重新载入数据集，申明定义的数据变换
test_set = mnist.MNIST('./data', train=False, transform=data_tf, download=True)
train_data = DataLoader(train_set, batch_size=64, shuffle=True)
test_data = DataLoader(test_set, batch_size=128, shuffle=False)

In [37]:
# 使用批标准化
class conv_bn_net(nn.Module):
    def __init__(self):
        super(conv_bn_net, self).__init__()
        self.stage1 = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=1),
            nn.BatchNorm2d(6),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2)
        )
        
        self.classfy = nn.Linear(400, 10)
    def forward(self, x):
        x = self.stage1(x)
        x = x.view(x.shape[0], -1)
        x = self.classfy(x)
        return x

net = conv_bn_net()
optimizer = torch.optim.SGD(net.parameters(), 1e-1) # 使用随机梯度下降，学习率 0.1

Assuming that the input to the nn.Conv2d layer is a 2D grayscale image with a height and width of H and W respectively, the output of the layer will have 6 channels and the same spatial dimensions as the input, i.e., the output will be a tensor of shape (1, 6, H, W).

The arguments to nn.Conv2d(1, 6, 3, padding=1) are:

1: the number of input channels. This means that the input to the layer has a single channel, i.e., it is a grayscale image.
6: the number of filters, which determines the number of output channels.
3: the size of the filters, i.e., the filters will be 3x3 in size.
padding=1: the amount of padding to add to the input on each side. A padding of 1 means that a one-pixel-wide border of zeros will be added around the input.
Therefore, if the input is of shape (1, 1, H, W), the output of the layer will be of shape (1, 6, H, W), where the height and width of the output will be the same as the height and width of the input, and the number of channels will be 6.

The values of x and y in nn.MaxPool2d(x, y) depend on the pooling kernel size and stride that we want to apply to the output of the previous layer, which in this case is the nn.BatchNorm2d layer.

Assuming that we want to apply 2x2 max pooling with a stride of 2, we can set x and y to 2,

The nn.MaxPool2d layer will downsample the output of the previous nn.BatchNorm2d layer by taking the maximum value in each non-overlapping 2x2 region of the feature maps, and moving the pooling window by 2 pixels vertically and horizontally. This results in a reduction in the spatial dimensions of the feature maps by a factor of 2.

Note that the pooling kernel size and stride can be set to different values, depending on the specific requirements of the neural network architecture and the input data.

In the given code snippet, self.classfy refers to a fully connected linear layer that will be used to classify the features extracted by the convolutional layers.

The nn.Linear(in_features, out_features) module takes the output of the convolutional layers as input and produces a tensor of shape (batch_size, out_features) as output. In this case, the input tensor to the linear layer will have a size of (batch_size, 400), where 400 corresponds to the number of output features of the last max pooling layer.

The second argument to nn.Linear is out_features, which determines the size of the output tensor. In this case, out_features is set to 10, which corresponds to the number of classes in the classification task.

Therefore, self.classfy = nn.Linear(400, 10) means that the output of the convolutional layers will be passed through a fully connected linear layer with 400 input features and 10 output features, which will produce the final classification scores for each input sample.

In [38]:
train(net, train_data, test_data, 5, optimizer, criterion)

<ipython-input-32-2d866a79c2c1>:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im, volatile=True)
<ipython-input-32-2d866a79c2c1>:56: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label, volatile=True)


Epoch 0. Train Loss: 0.154647, Train Acc: 0.957406, Valid Loss: 0.069137, Valid Acc: 0.977848, Time 00:00:21
Epoch 1. Train Loss: 0.059475, Train Acc: 0.982159, Valid Loss: 0.062075, Valid Acc: 0.979727, Time 00:00:23
Epoch 2. Train Loss: 0.047263, Train Acc: 0.985774, Valid Loss: 0.042184, Valid Acc: 0.986056, Time 00:00:24
Epoch 3. Train Loss: 0.039700, Train Acc: 0.988140, Valid Loss: 0.034782, Valid Acc: 0.987836, Time 00:00:24
Epoch 4. Train Loss: 0.034177, Train Acc: 0.989272, Valid Loss: 0.035976, Valid Acc: 0.987737, Time 00:00:24


In [39]:
# 不使用批标准化
class conv_no_bn_net(nn.Module):
    def __init__(self):
        super(conv_no_bn_net, self).__init__()
        self.stage1 = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2)
        )
        
        self.classfy = nn.Linear(400, 10)
    def forward(self, x):
        x = self.stage1(x)
        x = x.view(x.shape[0], -1)
        x = self.classfy(x)
        return x

net = conv_no_bn_net()
optimizer = torch.optim.SGD(net.parameters(), 1e-1) # 使用随机梯度下降，学习率 0.1    

In [40]:
train(net, train_data, test_data, 5, optimizer, criterion)

<ipython-input-32-2d866a79c2c1>:55: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  im = Variable(im, volatile=True)
<ipython-input-32-2d866a79c2c1>:56: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  label = Variable(label, volatile=True)


Epoch 0. Train Loss: 0.214514, Train Acc: 0.934668, Valid Loss: 0.097804, Valid Acc: 0.969541, Time 00:00:18
Epoch 1. Train Loss: 0.074727, Train Acc: 0.977279, Valid Loss: 0.058206, Valid Acc: 0.982397, Time 00:00:21
Epoch 2. Train Loss: 0.058251, Train Acc: 0.981760, Valid Loss: 0.064919, Valid Acc: 0.979529, Time 00:00:21
Epoch 3. Train Loss: 0.050116, Train Acc: 0.984092, Valid Loss: 0.057618, Valid Acc: 0.981507, Time 00:00:20
Epoch 4. Train Loss: 0.044248, Train Acc: 0.986141, Valid Loss: 0.056197, Valid Acc: 0.982298, Time 00:00:21


之后介绍一些著名的网络结构的时候，我们会慢慢认识到批标准化的重要性，使用 pytorch 能够非常方便地添加批标准化层